In [1]:
import pandas as pd
import sys
import re

In [2]:
sys.path.append('F:\Frederik\pfizer_mcrpc\paqc')

In [3]:
from paqc.report import report
from paqc.connectors import csv
from paqc.driver import driver
from paqc.utils import config_utils, utils

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
df_EMR = pd.read_excel(r'F:\Projects\Pfizer_mCRPC\Data\raw_data\Urology_EMR\Pfizer_mCRPC_UROEMR.xlsx')
df_pat_id = pd.read_excel(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Pfizer_mCRPC_Patients.xlsx')

In [11]:
df_pat_id.head()
df_EMR.head()

,PATIENT_ID,PN_FLAG,AGE_INDEX,AGE_PC,INDEX_DATE,LOOKBACK_DATE
0,377317624,N,85,85,2016-04-21,2014-04-22
1,73066609,N,80,78,2017-05-08,2015-05-09
2,70175556,N,50,50,2016-06-20,2014-06-21
3,713474282,N,83,81,2016-04-24,2014-04-25
4,1563563420,N,70,64,2014-05-14,2012-05-14


,PATIENT_ID,INDEX_DATE,DEIDCRPC2FLAG,DEIDCRPC2DATE,DEIDCRPC3FLAG,DEIDCRPC3DATE,DEIDMETASTATICFLAG,DEIDMETASTATICDATE,MET_DATE1,METAC_SITE_NM1,...,URO_FREE_TESTO_DATE7,URO_TNM_DATE,URO_LATEST_T,URO_LATEST_T_DATE,URO_LATEST_N,URO_LATEST_N_DATE,URO_LATEST_M,URO_LATEST_M_DATE,URO_LATEST_GLEASON,URO_LATEST_GLEASON_DATE
0,29658787,2014-07-18,1,2015-06-22,0,NaT,1,2015-08-10,2014-07-18,BONE & CONNECTIVE TISSUE CANCER,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,29674744,2015-05-14,1,2015-04-28,0,NaT,0,NaT,2015-05-14,BONE & CONNECTIVE TISSUE CANCER,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,29804745,2016-10-25,0,NaT,0,NaT,0,NaT,2016-10-25,OTHER CANCER,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,29929337,2016-12-01,0,NaT,0,NaT,1,2017-01-16,2016-12-01,BONE & CONNECTIVE TISSUE CANCER,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
4,29967237,2016-03-01,1,2015-02-18,1,2015-02-18,0,NaT,2016-03-01,LUNG CANCER,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


In [14]:
# merge with patient list to get lookback date and pn_flag
df_EMR = df_pat_id[['PATIENT_ID', 'PN_FLAG', 'LOOKBACK_DATE']].merge(df_EMR, on = 'PATIENT_ID', how = 'right')

## Merge urology with patient flag dataset

In [16]:
df_EMR['PN_FLAG'] = df_EMR['PN_FLAG'].map({'N': 0, 'P': 1})

In [22]:
df_EMR.head()

,PATIENT_ID,PN_FLAG,LOOKBACK_DATE,INDEX_DATE,DEIDCRPC2FLAG,DEIDCRPC2DATE,DEIDCRPC3FLAG,DEIDCRPC3DATE,DEIDMETASTATICFLAG,DEIDMETASTATICDATE,...,URO_FREE_TESTO_DATE7,URO_TNM_DATE,URO_LATEST_T,URO_LATEST_T_DATE,URO_LATEST_N,URO_LATEST_N_DATE,URO_LATEST_M,URO_LATEST_M_DATE,URO_LATEST_GLEASON,URO_LATEST_GLEASON_DATE
0,1563563420,0,2012-05-14,2014-05-14,1,2015-04-20,1,2016-07-20,0,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,1165766479,0,2012-02-05,2014-02-04,1,2013-01-04,0,NaT,1,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,91728018,0,2013-01-19,2015-01-19,0,NaT,0,NaT,0,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,240116111,0,2013-09-29,2015-09-29,0,NaT,0,NaT,0,NaT,...,NaT,2015-08-01,T2c,2015-08-01,NaN,NaT,NaN,NaT,NaN,NaT
4,72619119,0,2012-02-06,2014-02-05,0,NaT,0,NaT,1,NaT,...,NaT,2012-11-01,T3a,2012-11-01,NaN,NaT,NaN,NaT,NaN,NaT


In [23]:
df_EMR.to_csv(r'F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv', index=False)

## QC 

In [4]:
# Run the qcs again, with some extras
d = driver.Driver(r'F:\Frederik\pfizer_mcrpc\pfizer_mcrpc\raw_data_processing\QC\general_config_uro_EMR.yml')
d.run()

Loading config file...
Checking and parsing config file...
Config file checked and parsed. Starting QC pipeline...
-------------------------------------------------------
Starting QCs on input1, file path: F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv
Executing test qc9 on input1: F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv
Executing test qc9 on input1: F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv
Executing test qc9 on input1: F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv
Executing test qc9 on input1: F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv
-------------------------------------------------------
Generating HTML and CSV report...
-------------------------------------------------------


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
dict_config = config_utils.config_open(r'F:\Frederik\pfizer_mcrpc\pfizer_mcrpc\raw_data_processing\QC\general_config_uro_EMR.yml')[1]

In [7]:
df = csv.read_csv(dict_config, dict_config['general']['input1'])

In [9]:
df.head()

,PATIENT_ID,PN_FLAG,LOOKBACK_DATE,INDEX_DATE,DEIDCRPC2FLAG,DEIDCRPC2DATE,DEIDCRPC3FLAG,DEIDCRPC3DATE,DEIDMETASTATICFLAG,DEIDMETASTATICDATE,...,URO_FREE_TESTO_DATE7,URO_TNM_DATE,URO_LATEST_T,URO_LATEST_T_DATE,URO_LATEST_N,URO_LATEST_N_DATE,URO_LATEST_M,URO_LATEST_M_DATE,URO_LATEST_GLEASON,URO_LATEST_GLEASON_DATE
0,1563563420,0,2012-05-14,2014-05-14,1,2015-04-20,1,2016-07-20,0,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,1165766479,0,2012-02-05,2014-02-04,1,2013-01-04,0,NaT,1,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,91728018,0,2013-01-19,2015-01-19,0,NaT,0,NaT,0,NaT,...,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,240116111,0,2013-09-29,2015-09-29,0,NaT,0,NaT,0,NaT,...,NaT,2015-08-01,T2c,2015-08-01,NaN,NaT,NaN,NaT,NaN,NaT
4,72619119,0,2012-02-06,2014-02-05,0,NaT,0,NaT,1,NaT,...,NaT,2012-11-01,T3a,2012-11-01,NaN,NaT,NaN,NaT,NaN,NaT


In [12]:
ss_diff = (df['INDEX_DATE'] - df['DEIDMETASTATICDATE']).dt.days

In [15]:
ss_after_index = ss_diff[(ss_diff < 0)]

In [17]:
min(ss_after_index)

-1256.0